<span style='font-family:"Times New Roman"'><font size="3">ML-zoomcamp Data talks cours 5 : </font></span>
<br />
<br />
<br />
<br />
<span style='font-family:"Times New Roman"'><div align="center"><font size="6">**Déploiement**</font></div></span>
<br /><br />
<span style='font-family:"Times New Roman"'><font size="5"><div align="right">*9 - 16 oct 2023*</div></font></span>
<br />
[cours](https://github.com/DataTalksClub/machine-learning-zoomcamp/tree/master/05-deployment)<br />
[ennoncé du devoir](https://github.com/DataTalksClub/machine-learning-zoomcamp/blob/master/cohorts/2023/05-deployment/homework.md)</font></span></div>
<br />
_____
<span style='color:blue'></span>
<br />
<br />

## Q1

In [ ]:
pipenv --version

## Q2

In [ ]:
pipenv install Scikit-Learn==1.3.1

## Q3

In [8]:
import pickle
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression

In [4]:
model_file = "model1.bin"
dv_file = "dv.bin"

with open(model_file, 'rb') as model:
    model = pickle.load(model)
with open(dv_file, 'rb') as dv:
    dv = pickle.load(dv)

C:\Users\laara\anaconda3\envs\ml-zoomcamp\Lib\site-packages\sklearn\base.py:347: InconsistentVersionWarning: Trying to unpickle estimator LogisticRegression from version 1.3.1 when using version 1.3.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
C:\Users\laara\anaconda3\envs\ml-zoomcamp\Lib\site-packages\sklearn\base.py:347: InconsistentVersionWarning: Trying to unpickle estimator DictVectorizer from version 1.3.1 when using version 1.3.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [5]:
model

LogisticRegression()

In [6]:
dv

DictVectorizer(sparse=False)

In [29]:
client = {"job": "retired", "duration": 445, "poutcome": "success"}

In [31]:
X = dv.transform([client])
y_pred = model.predict_proba(X)[0, 1]
credit_acceptation = model.predict(X)

print('proba ', y_pred, credit_acceptation)

proba  0.9019309332297606 ['yes']


## Q4

In [ ]:
sudo apt install pipenv
pipenv install flask scikit-learn numpy

In [ ]:
# script flask.py
import pickle
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction import DictVectorizer
from flask import Flask
from flask import request
from flask import jsonify

# Define your app name here
app=Flask('credit_acceptation')

# Load the model and dv
model_file = "model1.bin"
dv_file = "dv.bin"
with open(model_file, 'rb') as model:
    model = pickle.load(model)
with open(dv_file, 'rb') as dv:
    dv = pickle.load(dv)

# Use this decorator: this will be the predict that gives you the output of a request
@app.route('/predict', methods=['POST'])
def predict():
    # Get your customer informations
    client= request.get_json()
    
    # Make your prediction using the loaded model and dv
    X = dv.transform([client])
    y_pred = model.predict_proba(X)[0,1]
    acceptation = y_pred >= 0.5
    
    # Obtain the result: you have to specify the type of the variables
    result = {
        'credit_acceptation_probability': float(y_pred),
        'credit_acceptation': bool(credit_acceptation)
    }
    
    # Return your result in a json format
    return jsonify(result)

# Here, when you run your script with python in dev environment, you can debug it
if __name__=="__main__":
    app.run(debug=True, host='0.0.0.0', port=9696)

In [ ]:
pipenv shell
python3 flask_script.py

In [ ]:
pipenv install requests

In [ ]:
# test avec flask test.py
import requests

# Here is your API url:
url = 'http://localhost:9696/predict'

# Here are your customer characteristics, in a dict:
client = {"job": "unknown", "duration": 270, "poutcome": "failure"}

# Here your get your response from your API using the customer above
response = requests.post(url, json=client).json()
print(response)

# Hereyou precise if a promotional email should be sent to this customer
if response['credit_acceptation'] == True:
    print('accepter le crédit')
else:
    print('refuser le crédit')

In [ ]:
python3 predict_test.py

In [32]:
client = {"job": "unknown", "duration": 270, "poutcome": "failure"}

In [34]:
X = dv.transform([client])
y_pred = model.predict_proba(X)[0, 1]
credit_acceptation = model.predict(X)

print('proba ', y_pred, credit_acceptation)

proba  0.13968947052356817 ['no']


## Q5

In [ ]:
! curl -s https://hub.docker.com/v2/repositories/svizor/zoomcamp-model/tags/ | \
    jq '.results[] | select(.name=="3.10.12-slim") | .full_size'

## Q6

In [ ]:
pipenv install gunicorn # sous windows ce sera waitress

In [ ]:
# pour faire tourner l'api dans le nouvel environnement
gunicorn --bind 0.0.0.0:9696 app:app

In [ ]:
# fichier docker
FROM svizor/zoomcamp-model:3.10.12-slim

RUN pip install pipenv

WORKDIR /app

COPY ["Pipfile", "Pipfile.lock", "./"]

RUN pipenv install --system --deploy

COPY ["app.py", "./"]

EXPOSE 9696

ENTRYPOINT ["gunicorn", "--bind=0.0.0.0:9696", "app:app"]

In [ ]:
# construire l'image
docker build -t zoomcamp-test .

In [ ]:
# run the container
docker run -it --rm -p 9696:9696 zoomcamp-test

In [35]:
# avec model1
client = {"job": "retired", "duration": 445, "poutcome": "success"}

In [36]:
X = dv.transform([client])
y_pred = model.predict_proba(X)[0, 1]
credit_acceptation = model.predict(X)

print('proba ', y_pred, credit_acceptation)

proba  0.9019309332297606 ['yes']
